# Mapping 2
- Line of Interest (Circle)
- Intensity mapping
- Linear interpolation

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import Audio
from scipy.io.wavfile import write
from datetime import datetime
from pya import *

%matplotlib widget

In [ ]:
def get_interpolated(array, index):
    if not hasattr(array, "__len__"): return array # if scalar
    return (1 - (index[0] % 1)) * get_interpolated(array[int(np.floor(index[0]))], index[1:]) + (index[0] % 1) * get_interpolated(array[int(np.ceil(index[0]))], index[1:])

In [ ]:
# Doesnt Work!
def vectorized_interpolation(array, indices): # indices.shape (dim_array, desired number of interpolations)
    if len(indices) == 0: return array
    t = indices[0] % 1
    floor_indices = np.floor(indices[0]).astype(int)
    ceil_indices  = np.ceil (indices[0]).astype(int)
    return (1 - t) * vectorized_interpolation(array[floor_indices], indices[1:]) + t * vectorized_interpolation(array[ceil_indices], indices[1:])

In [ ]:
#vectorized_interpolation(np.array([[0, 1, 2], [10, 20, 30]]), np.array([[0.5, 1], [0.5, 1]]))

# Simulation

In [ ]:
import simulation

n = 128
sim_speed = 0.004
sim_fps = 400
duration = 6
frame_amount = duration * sim_fps

initial_state = np.array([[simulation.gaussian_impulse(x, y, n, offset=[-0.6, 0.0], width=0.15, impulse=0.07) for x in range(n)] for y in range(n)])

#potential = np.array([[simulation.parabola(x, y, n, offset=[0, 0], factor=10000) for x in range(n)] for y in range(n)])
potential = np.zeros((n, n))

multi_slit = [(-15, -13), (-8, -6), (-1, 1), (6, 8), (13, 15)]
double_slit = [(-4, -2), (2, 4)]
single_slit = [(-2, 2)]
slits = double_slit

frames = simulation.sim(n, sim_fps, duration, slits, sim_speed, initial_state=initial_state, potential=potential, normalize=True)

In [ ]:
plt.figure()
plt.plot(np.sum(np.square(np.abs(frames)), axis=(1, 2))[:5])

In [ ]:
plt.pcolormesh(pow(np.abs(frames[0]), 2.0/3.0), cmap='inferno', vmin=0, vmax=1)
plt.pcolormesh(potential, vmin=0, vmax=20000)
plt.colorbar()
plt.show()

# Video

In [ ]:
import video

# save video
video_filename, anim = video.create(frames, 20, frame_amount, sim_fps, slits, n)

# plt.close()

# Sonification

In [ ]:
# circle of interest
def circle(radius, center):
    radians_per_sample = 2 * np.pi * frequency / sample_rate
    radians = np.arange(sample_rate * duration) * radians_per_sample
    x = (0.5 * n * (radius * np.cos(radians) + 1 + center[0]))#.astype(int)
    y = (0.5 * n * (radius * np.sin(radians) + 1 + center[1]))#.astype(int)
    return x, y

# line of interest
def line(start, end):
    t = np.arange(sample_rate * duration) / sample_rate * frequency % 1
    x = start[0]+n//2 + t * (end[0] - start[0])
    y = start[1]+n//2 + t * (end[1] - start[1])
    return x, y

In [ ]:
# parameters
sample_rate = 44100
frequency = 220

# choose method
x, y = circle(radius=0.25, center=[0, 0])
# x, y = line(start=[-60, 0], end=[60, 0])



In [ ]:
# Sonification
frames_indices = np.linspace(0, frame_amount - 1, sample_rate * duration, endpoint=False)

# No interpolation
#audio = frames[frames_indices.astype(int), y.astype(int), x.astype(int)]
#audio = np.square(np.abs(audio))

# Hardcoded interpolation
# Way faster than any other interpolation!
t = np.array([frames_indices, y, x]) % 1
floors = np.floor([frames_indices, y, x]).astype(int)
ceils  = np.ceil ([frames_indices, y, x]).astype(int)

audio = ((1-t[0]) * ((1-t[1]) * ((1-t[2]) * np.square(np.abs(frames[floors[0], floors[1], floors[2]]))
                              +     t[2]  * np.square(np.abs(frames[floors[0], floors[1], ceils [2]])))
                  +     t[1]  * ((1-t[2]) * np.square(np.abs(frames[floors[0], ceils [1], floors[2]]))
                              +     t[2]  * np.square(np.abs(frames[floors[0], ceils [1], ceils [2]]))))
          + t[0]  * ((1-t[1]) * ((1-t[2]) * np.square(np.abs(frames[ceils [0], floors[1], floors[2]]))
                              +     t[2]  * np.square(np.abs(frames[ceils [0], floors[1], ceils [2]])))
                  +     t[1]  * ((1-t[2]) * np.square(np.abs(frames[ceils [0], ceils [1], floors[2]]))
                              +     t[2]  * np.square(np.abs(frames[ceils [0], ceils [1], ceils [2]])))))

#audio = np.square(np.abs(audio))

print("Finished sonification")

In [ ]:
fade = 0.005
if fade > 0:
    fade_samples = int(sample_rate * fade)
    audio[:fade_samples]  *= np.square(np.linspace(start=0, stop=1, num=fade_samples, endpoint=False))
    audio[-fade_samples:] *= np.square(np.linspace(start=1, stop=0, num=fade_samples, endpoint=False))
audio_filename = f'output/sonification_{datetime.now().strftime("%Y_%m_%d-%H_%M_%S")}.wav'
write(audio_filename, sample_rate, np.round((audio - np.average(audio)) / np.max(np.abs(audio - np.average(audio))) * 32767).astype(np.int16))
print(f"Sonification saved as {audio_filename}")
Audio(audio, rate=sample_rate)

In [ ]:
device_info();

In [ ]:
s = startup(device=1)

In [ ]:
a1 = Asig(audio, sr=sample_rate)

In [ ]:
plt.figure()
a1.iirfilter(30, order=2, btype='highpass').fade_in(0.005).fade_out(0.005).norm().stereo().play(onset=1, rate=2) # rate: resampling (oktave hoch/runter)
plt.close()

In [ ]:
# weitere feature: filter, nicht-linearer transfer, multichannel (stereo)

# time slice
a1[{2.0:4.0}].play()

# audio hinten anhängen
a1.x[-1:] = a1[{1.0:2.0}]

# audio überlagern (wie genau? vielleicht wie oben aber mit +=)

# advanced access modes
a1.x # extend(able bounds)
a1.b # bound(s shrinkable)
a1.o # overwrite

# spectrogram (nperseg für freq vs zeitauflösung)
a1.to_stft(nperseg=4096).plot(ylim=(0, 2000), scale='log')

# get
numpy_array_of_audio = a1.sig

# Klassen
#Asig Aspec Astft Arecorder Aserver
#Ugen

# load file
Asig('filename.wav')

In [ ]:
plt.figure()
a1.to_stft(nperseg=4096).plot()

In [ ]:
plt.figure()
plt.plot(audio)

# Combine Video & Audio

In [ ]:
# combine audio & video
video.combine(audio_filename, video_filename)